In [27]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report


In [28]:
# Load train & test sets
train_df = pd.read_csv("UNSW_NB15_training-set.csv")
test_df  = pd.read_csv("UNSW_NB15_testing-set.csv")

train_df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [29]:
# Identify categorical and numerical columns **before encoding**
categorical_cols = train_df.select_dtypes(include=['object']).columns
numeric_cols = train_df.columns.difference(categorical_cols.union(['label']))


In [30]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', StandardScaler(), numeric_cols)
])


In [31]:

# Base estimator for bagging (Decision Tree)
base_tree = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,           
    min_samples_split=4,
    min_samples_leaf=2,
    class_weight="balanced"
)


In [32]:
# Bagging Classifier
model = BaggingClassifier(
    base_estimator=base_tree,
    n_estimators=150,
    max_samples=0.8,
    max_features=1.0,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)


In [33]:
# Build full pipeline
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', model)
])

In [34]:
# Train
pipeline.fit(train_df.drop('label', axis=1), train_df['label'])

# Predict
y_pred = pipeline.predict(test_df.drop('label', axis=1))

# Evaluate
print(classification_report(test_df['label'], y_pred))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37000
           1       1.00      1.00      1.00     45332

    accuracy                           1.00     82332
   macro avg       1.00      1.00      1.00     82332
weighted avg       1.00      1.00      1.00     82332
